Perbedaan NLTK dan Spacy
Spacy is better than NLTK in terms of performance.Here, there are some comparison

1- NLTK is a string processing library. It takes strings as input and returns strings or lists of strings as output.Whereas, spaCy uses object-oriented approach.When we parse a text, spaCy returns document object whose words and sentences are objects themselves.

2- spaCy has support for word vectors whereas NLTK does not.

3- In word tokenization and POS-tagging spaCy performs better, but in sentence tokenization, NLTK outperforms spaCy.

4- NLTK supports various languages whereas spaCy have statistical models for 7 languages (English, German, Spanish, French, Portuguese, Italian, and Dutch). It also supports named entities for multi language.
                                                                        

NLTK

In [1]:
# token
import nltk
from nltk.tokenize import word_tokenize 

def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [2]:
teks_nltk = word_tokenize_wrapper('Yahya beserta teman-teman TK suka melihat lumba-lumba di Batang Dolphin Center')

In [3]:
teks_nltk

['Yahya',
 'beserta',
 'teman-teman',
 'TK',
 'suka',
 'melihat',
 'lumba-lumba',
 'di',
 'Batang',
 'Dolphin',
 'Center']

spacy
lakukan installasi dan update lib spacy

In [4]:
from spacy.lang.id import Indonesian
import spacy

nlp = Indonesian()  # use directly
nlp = spacy.blank('id')  # blank instance'

In [5]:
Teks = nlp('Yahya beserta teman-teman TK suka melihat lumba-lumba di Batang Dolphin Center')

In [6]:
Token_kata = [token.text for token in Teks]

In [7]:
Token_kata

['Yahya',
 'beserta',
 'teman-teman',
 'TK',
 'suka',
 'melihat',
 'lumba-lumba',
 'di',
 'Batang',
 'Dolphin',
 'Center']

load library

In [8]:
import re
import string
import time
from copy import deepcopy

load dataset

In [13]:
import pandas as pd
import json

In [14]:
file = "2018-01-01/01/00/1514764800000.republika.387b7eb869.json"

In [15]:
with open(file) as file:
    data = json.load(file)

In [16]:
df = pd.json_normalize(data)

In [17]:
df

,url,date,title,content
0,http://khazanah.republika.co.id/berita/dunia-i...,2018-01-01 00:00:00,Dzikir Nasional Siapkan Spiritual untuk Lebih ...,"REPUBLIKA.CO.ID, JAKARTA -- Dzikir Nasional 20..."


In [18]:
df = df.drop(['url'], axis=1)

In [19]:
df = df.drop(['date'], axis=1)

In [20]:
df

,title,content
0,Dzikir Nasional Siapkan Spiritual untuk Lebih ...,"REPUBLIKA.CO.ID, JAKARTA -- Dzikir Nasional 20..."


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1 non-null      object
 1   content  1 non-null      object
dtypes: object(2)
memory usage: 148.0+ bytes


Preprocessing dataset

kita akan pakai modul dari AMS 01-03

disamping itu kita akan memakai modul https://github.com/cbaziotis/ekphrasis

In [22]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


C:\Python311\Lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


C:\Python311\Lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [23]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [24]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('rt'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text)

In [25]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

lakukan pembersihan dengan memanggil fungsi yang didefinisikan diatas

In [27]:
i = 0
final_string = []
s = ""
for text in df['content'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = lower(text)
    proc = change_stripe(text)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    #proc = remove_rt(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [28]:
df["step01"] = final_string

In [29]:
df.head(10)

,title,content,step01
0,Dzikir Nasional Siapkan Spiritual untuk Lebih ...,"REPUBLIKA.CO.ID, JAKARTA -- Dzikir Nasional 20...",republika co id jakarta dzikir nasional bert...


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1 non-null      object
 1   content  1 non-null      object
 2   step01   1 non-null      object
dtypes: object(3)
memory usage: 156.0+ bytes


In [31]:
df_hapus = df[~df['step01'].str.contains(" ")]

In [32]:
df_hapus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    0 non-null      object
 1   content  0 non-null      object
 2   step01   0 non-null      object
dtypes: object(3)
memory usage: 0.0+ bytes


In [33]:
df_hapus.head(10)

,title,content,step01


In [34]:
df_new = df[~df.isin(df_hapus)].dropna()

In [35]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1 non-null      object
 1   content  1 non-null      object
 2   step01   1 non-null      object
dtypes: object(3)
memory usage: 156.0+ bytes


In [36]:
df_new

,title,content,step01
0,Dzikir Nasional Siapkan Spiritual untuk Lebih ...,"REPUBLIKA.CO.ID, JAKARTA -- Dzikir Nasional 20...",republika co id jakarta dzikir nasional bert...


normalisasi kata slang menjadi kata baku

In [37]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [38]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [39]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [40]:
df_new.head(10)

,title,content,step01,tokens
0,Dzikir Nasional Siapkan Spiritual untuk Lebih ...,"REPUBLIKA.CO.ID, JAKARTA -- Dzikir Nasional 20...",republika co id jakarta dzikir nasional bert...,"[republika, co, id, jakarta, dzikir, nasional,..."


In [41]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [42]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

C:\Users\mmahf\AppData\Local\Temp\ipykernel_3380\2408214049.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
C:\Users\mmahf\AppData\Local\Temp\ipykernel_3380\2408214049.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [43]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [44]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [45]:
df_new["step02"] = final_string_tokens

In [46]:
df_new.head(10)

,title,content,step01,tokens,final_tokens,step02
0,Dzikir Nasional Siapkan Spiritual untuk Lebih ...,"REPUBLIKA.CO.ID, JAKARTA -- Dzikir Nasional 20...",republika co id jakarta dzikir nasional bert...,"[republika, co, id, jakarta, dzikir, nasional,...","[republika, co, id, jakarta, dzikir, nasional,...",republika co id jakarta dzikir nasional bertuj...


In [47]:
df.to_csv('clean_dataset2.csv',sep=";")